`# Import dependencies`

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
from scipy import stats 

`# Get stock market data`

In [2]:
end = dt.datetime.now()
start = dt.datetime(2015,1,1)

yf.pdr_override()
df = pdr.get_data_yahoo(["^SPX", "NVDA", "GOOG", "META", "TSLA"], start, end)

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


[*********************100%%**********************]  5 of 5 completed


In [3]:
log_returns = np.log(df.Close/df.Close.shift(1)).dropna()
log_returns.head()

Ticker,GOOG,META,NVDA,TSLA,^SPX
Date,,,,,
2015-01-05,-0.021066,-0.016191,-0.017034,-0.042950,-0.018447
2015-01-06,-0.023450,-0.013565,-0.030787,0.005648,-0.008933
2015-01-07,-0.001715,0.000000,-0.002609,-0.001563,0.011563
2015-01-08,0.003148,0.026309,0.036927,-0.001566,0.017730
2015-01-09,-0.013035,-0.005644,0.004020,-0.018981,-0.008439


`# Directly calculate beta`

In [4]:
def calc_beta(df):
    # Rearrange columns
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    np_array = df.values
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            s = np_array[:, ind]
            covariance = np.cov(s,m)
            beta.append(covariance[0,1]/covariance[1,1])
    return pd.Series(beta, df.columns[1:], name = 'Beta')

In [5]:
calc_beta(log_returns)

Ticker
GOOG    1.142916
META    1.276644
NVDA    1.709287
TSLA    1.462314
Name: Beta, dtype: float64

`# Use linear regression to get coefficient of market and stocks returns`

In [6]:
def regression_beta(df):
    # Rearrange columns
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    np_array = df.values
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            s = np_array[:, ind]
            beta.append(stats.linregress(m,s)[0])
    return pd.Series(beta, df.columns[1:], name = 'Beta')

In [7]:
regression_beta(log_returns)

Ticker
GOOG    1.142916
META    1.276644
NVDA    1.709287
TSLA    1.462314
Name: Beta, dtype: float64

`# Use matrix algebra to compute regression in one line`

In [8]:
def matrix_beta(df):
    # Rearrange columns
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    X = df.values[:, [0]]
    X = np.concatenate([np.ones_like(X), X], axis = 1)
    beta = np.linalg.pinv(X.T @ X) @ X.T @ df.values[:, 1:]
    return pd.Series(beta[1], df.columns[1:], name='Beta')

In [9]:
beta = matrix_beta(log_returns)

`# Define your portfolio and make DataFrame`

In [10]:
units = np.array([100, 250, 200, 350])
stocks = df.Close[-1:].values.tolist()[0]
price = np.array([round(price, 2) for price in stocks[:4]])
value = units*price
weight = [round(val/sum(value), 2) for val in value]
beta = round(beta, 2)

In [11]:
stocklist = ["GOOG", "META", "NVDA", "TSLA"]
Portfolio = pd.DataFrame({
    'Stock': stocklist,
    'Direction': 'Long',
    'Type': 'S',
    'Stock Price': price,
    'Price': price,
    'Units': units,
    'Value': value,
    'Weights': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Weights,Beta,Weighted Beta
Ticker,,,,,,,,,,
GOOG,GOOG,Long,S,183.60,183.60,100,18360.0,0.07,1.14,0.0798
META,META,Long,S,488.69,488.69,250,122172.5,0.49,1.28,0.6272
NVDA,NVDA,Long,S,122.59,122.59,200,24518.0,0.10,1.71,0.1710
TSLA,TSLA,Long,S,246.38,246.38,350,86233.0,0.34,1.46,0.4964


`# Options: let's consider things in terms of Delta`

In [12]:
Portfolio = Portfolio.drop(['Weighted Beta', 'Weights'], axis = 1)
Portfolio['Delta'] = Portfolio['Units']
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Ticker,,,,,,,,,
GOOG,GOOG,Long,S,183.60,183.60,100,18360.0,1.14,100
META,META,Long,S,488.69,488.69,250,122172.5,1.28,250
NVDA,NVDA,Long,S,122.59,122.59,200,24518.0,1.71,200
TSLA,TSLA,Long,S,246.38,246.38,350,86233.0,1.46,350


`# Add options to portfolio`

In [16]:
Options = [{'option': 'NVDA240726C00050000', 'underlying': 'NVDA', 'price': 122.59, 'units': 5, 'delta': 1.00, 'direction': 'Short', 'type': 'Call'},
          {'option': 'GOOG240726C00090000', 'underlying': 'GOOG', 'price': 183.60, 'units': 3, 'delta': -1.00, 'direction': 'Long', 'type': 'Put'}]

In [20]:
for index, row in enumerate(Options):
    Portfolio.loc[row['option']] = [row['underlying'], row['direction'], row['type'], Portfolio.loc[row['underlying'], 'Price'],
                                    row['price'], row['units'], row['price']*row['units']*100, beta[row['underlying']],
                                    (row['delta']*row['units']* 100 if row['direction'] == 'Long' else -row['delta']*row['units']*100)]
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Ticker,,,,,,,,,
GOOG,GOOG,Long,S,183.60,183.60,100,18360.0,1.14,100.0
META,META,Long,S,488.69,488.69,250,122172.5,1.28,250.0
NVDA,NVDA,Long,S,122.59,122.59,200,24518.0,1.71,200.0
TSLA,TSLA,Long,S,246.38,246.38,350,86233.0,1.46,350.0
NVDA240726C00050000,NVDA,Short,Call,122.59,122.59,5,61295.0,1.71,-500.0
GOOG240726C00090000,GOOG,Long,Put,183.60,183.60,3,55080.0,1.14,-300.0


`#  Weight the Delta's using Beta`

In [38]:
Portfolio['^SPX Weighted Delta (point)'] = round(Portfolio['Beta'] * Portfolio['Stock Price'] * Portfolio['Delta'] * 1/stocks[4],2)
Portfolio['^SPX Weighted Delta (1%)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']) * Portfolio['Delta'] * 1/100,2)
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta,^SPX Weighted Delta (point),^SPX Weighted Delta (1%)
Ticker,,,,,,,,,,,
GOOG,GOOG,Long,S,183.60,183.60,100,18360.0,1.14,100.0,3.77,209.30
META,META,Long,S,488.69,488.69,250,122172.5,1.28,250.0,28.15,1563.81
NVDA,NVDA,Long,S,122.59,122.59,200,24518.0,1.71,200.0,7.55,419.26
TSLA,TSLA,Long,S,246.38,246.38,350,86233.0,1.46,350.0,22.66,1259.00
NVDA240726C00050000,NVDA,Short,Call,122.59,122.59,5,61295.0,1.71,-500.0,-18.87,-1048.14
GOOG240726C00090000,GOOG,Long,Put,183.60,183.60,3,55080.0,1.14,-300.0,-11.30,-627.91


`# Total the Delta's to get Portfolio Overview`

In [42]:
Portfolio.loc['Total', ['Value', '^SPX Weighted Delta (point)', '^SPX Weighted Delta (1%)']] = Portfolio[['Value','^SPX Weighted Delta (point)', '^SPX Weighted Delta (1%)']].sum()
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta,^SPX Weighted Delta (point),^SPX Weighted Delta (1%)
Ticker,,,,,,,,,,,
GOOG,GOOG,Long,S,183.60,183.60,100.0,18360.0,1.14,100.0,3.77,209.30
META,META,Long,S,488.69,488.69,250.0,122172.5,1.28,250.0,28.15,1563.81
NVDA,NVDA,Long,S,122.59,122.59,200.0,24518.0,1.71,200.0,7.55,419.26
TSLA,TSLA,Long,S,246.38,246.38,350.0,86233.0,1.46,350.0,22.66,1259.00
NVDA240726C00050000,NVDA,Short,Call,122.59,122.59,5.0,61295.0,1.71,-500.0,-18.87,-1048.14
GOOG240726C00090000,GOOG,Long,Put,183.60,183.60,3.0,55080.0,1.14,-300.0,-11.30,-627.91
Total,NaN,NaN,NaN,NaN,NaN,NaN,367658.5,NaN,NaN,31.96,1775.32
